- https://verl.readthedocs.io/en/latest/examples/config.html

In [1]:
from IPython.display import Image

- 带 kl 散度惩罚的强化学习目标
    - dpo (RL Fine-Tuning Phase)

$$
\mathcal{J}(\pi_\theta) = \mathbb{E}_{\mathbf{q} \sim p_Q} \left[ \mathbb{E}_{\mathbf{o} \sim \pi_\theta(\cdot|\mathbf{q})} [R(\mathbf{q}, \mathbf{o})] - \beta D_{KL}[\pi_\theta(\cdot|\mathbf{q}) || \pi_{\text{ref}}(\cdot|\mathbf{q})] \right]
$$

- PPO-Clip 目标函数

$$
\mathcal{J}_{\text{PPO}}(\pi_\theta) = \mathbb{E}_{\mathbf{q} \sim p_Q, \mathbf{o} \sim \pi_{\theta_{\text{old}}}(\cdot|\mathbf{q})} \left[ \sum_{t=1}^{|\mathbf{o}|} \min \left[ \frac{\pi_\theta(o_t|\mathbf{q}, \mathbf{o}_{<t})}{\pi_{\theta_{\text{old}}}(o_t|\mathbf{q}, \mathbf{o}_{<t})} \hat{A}_t, \text{clip}\left(\frac{\pi_\theta(o_t|\mathbf{q}, \mathbf{o}_{<t})}{\pi_{\theta_{\text{old}}}(o_t|\mathbf{q}, \mathbf{o}_{<t})}, 1-\epsilon, 1+\epsilon\right) \hat{A}_t \right] \right]
$$


- 公式1 是一个高层次的、理论上的目标。它告诉我们“我们想要什么”：一个高奖励且行为正常的模型。
- 公式2 是一个具体的、可操作的算法。它告诉我们“我们具体怎么做”：通过PPO算法稳定地更新模型参数以达成目标。

## pg loss

In [3]:
Image(url='https://huggingface.co/blog/assets/93_deep_rl_ppo/recap.jpg', width=400)

https://huggingface.co/blog/deep-rl-ppo
$$
L^{PPO}=E_{t}[\max(-r_t(\theta)\hat A_t, -\text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon)\hat A_t)]
$$

- 当 $\hat A_t\gt 0$（好动作）时，不希望 $r_t(\theta)$ 过大，将其限制在 $1+\epsilon$ 之内
    - $r_t(\theta) > 1+\epsilon$
    - $\pi_\theta(a_t|s_t) >> \pi_{\theta_{old}}(a_t|s_t)$
- 当 $\hat A_t\lt 0$（好动作）时，不希望 $r_t(\theta)$ 过小，将其限制在 $1-\epsilon$ 之内
    - $r_t(\theta) < 1-\epsilon$
- 还需要注意 gradient 的部分；
    - 什么情况下不学习（不更新模型）
    - 方向盘（梯度） vs. 放大器
- sign of objective is sign of $\hat A_t$
    - 正 objective：$\hat A_t\gt 0$，增大 $\pi_\theta(a_t|s_t)$
    - 负 objective：$\hat A_t\lt 0$，降低 $\pi_\theta(a_t|s_t)$
- $\hat A_t$: 不贡献梯度（是一个不在计算图中的标量）
    - $\hat A_t$ 的计算来源于 $\pi_{\theta_{old}}$（$\pi_{\theta_{old}$ 在一次 ppo epoch 中只负责 generation 与采集数据）

### pg loss curve

- policy_loss 的正负直接反映了在一个训练批次（batch）中，智能体所采取的行动的平均质量（相对于价值函数的评估）。
- policy_loss < 0 (负数)代表什么？
    - 这意味着 L_CLIP(θ) 是正数。
    - 深层含义？ 在这个批次中，“好”的动作（$\hat A_t>0$）所带来的正面影响，超过了“坏”的动作（$\hat A_t < 0$）带来的负面影响。简单来说，智能体在这个批次中的平均表现超出了它自己的预期（价值函数 $V(s_t)$）。
优化方向？ 优化器会试图最小化这个负数，即让它变得更负。这会进一步增强那些“好”动作的概率。
    - 是好是坏？
        - 通常是好兆头：表明智能体正在有效地探索并找到了能带来更高回报的策略。这是学习在正常发生的信号。
    - 潜在风险：如果 policy_loss 持续保持在非常大的负值，可能意味着价值函数（Critic）的更新跟不上策略（Actor）的提升。Critic 总是低估当前策略的价值，导致 Advantage 持续偏高。这可能预示着潜在的不稳定。
- policy_loss > 0 (正数)
    - 代表什么？ 这意味着 L_CLIP(θ) 是负数。
    - 深层含义？ 在这个批次中，“坏”的动作所带来的负面影响，超过了“好”的动作带来的正面影响。智能体的平均表现低于它自己的预期。
    - 优化方向？ 优化器会试图最小化这个正数，即让它向 0 靠近。这会削弱那些“坏”动作的概率。
    - 是好是坏？
        - 这是学习的正常组成部分：智能体需要通过试错来学习。采取了坏的动作，然后通过正的 policy_loss 来惩罚这些动作，这是完全正常的。
        - 危险信号：如果 policy_loss 持续保持在较高的正值，这是一个非常糟糕的信号。它意味着智能体在不断地做出比自己预期还要差的决策，策略可能正在恶化或完全没有学到东西。
- 理想的趋势：在 0 附近震荡，无明显上升或下降趋势
    - 一个完美的策略和价值函数组合，意味着对于任何状态，价值函数都能准确预测期望回报。因此，任何动作的优势函数 $\hat A_t$ 都会趋近于 0。
    - 在实际训练中，策略和价值函数是交替迭代、相互追赶的。策略稍微变好一点 (policy_loss 变负)，价值函数马上学习跟上，将 policy_loss 拉回到 0 附近。策略尝试了坏动作 (policy_loss 变正)，然后修正自己，又回到 0 附近。
    - 稳定学习：Actor 和 Critic 步调一致，策略在信赖域内被稳定地优化。
    - 有效基线：价值函数提供了一个准确的基线（baseline），使得优势函数的估计是有意义的。

### grpo pg loss

- $\sum_{i,A_i>0}r_{i,t}A_{i,t} + \sum_{j,A_j<0}r_{j,t}A_{j,t}>0$
    - $r_{i,t}>1, r_{j,t}\lt 1$
- $\sum_{i,A_i>0}r_{i,t}A_{i,t} + \sum_{j,A_j<0}r_{j,t}A_{j,t}<0$
    - $r_{i,t}<1, r_{j,t}> 1$

### dual-clip ppo

In [2]:
Image(url='./imgs/dual-clip.png', width=300)

- Dual-Clip PPO
    - https://arxiv.org/pdf/1912.09729
 
$$
L^{\text{dual-clip}}=\min(L^{PPO}, -c\cdot \hat A_t)
$$

### entropy

- actor
    - entropy_coeff: 0.0 (default)
```python
# entropy = verl_F.entropy_from_logits(logits)  # (bsz, response_length)
policy_loss = pg_loss - entropy_loss * entropy_coeff
```

```python
def entropy_from_logits(logits: torch.Tensor):
    """Calculate entropy from logits."""
    pd = torch.nn.functional.softmax(logits, dim=-1)
    entropy = torch.logsumexp(logits, dim=-1) - torch.sum(pd * logits, dim=-1)
    return entropy
```

### use_kl_loss

```python
policy_loss = policy_loss + kl_loss * self.config.kl_loss_coef
```

### agg_loss

- $L\in R^{B\times T}$ 表示 loss mat, $M\in {\{0,1\}}^{B\times T}$ 表示损失掩码 (loss_mask)（为 1 表示损失计算在内）
    - $ \mathcal{L}_{\text{token-mean}} = \frac{\sum_{i=1}^{B} \sum_{j=1}^{T} L_{i,j} \cdot M_{i,j}}{\sum_{i=1}^{B} \sum_{j=1}^{T} M_{i,j}} $
    - $\mathcal{L}_{\text{seq-mean-token-sum}} = \frac{1}{B} \sum_{i=1}^{B} \left( \sum_{j=1}^{T} L_{i,j} \cdot M_{i,j} \right)$
    - $\mathcal{L}_{\text{seq-mean-token-mean}} = \frac{1}{B} \sum_{i=1}^{B} \left( \frac{\sum_{j=1}^{T} L_{i,j} \cdot M_{i,j}}{\sum_{j=1}^{T} M_{i,j}} \right)$
    - $\mathcal{L}_{\text{seq-mean-token-sum-norm}} = \frac{\sum_{i=1}^{B} \sum_{j=1}^{T} L_{i,j} \cdot M_{i,j}}{T} $